In [1]:
import pandas as pd
import numpy as np
import re
import os

#from sklearn.feature_extraction.text import CountVectorizer
#from konlpy.tag import Okt
#okt = Okt()

In [2]:
#1. Extract File paths and names(only name of that file) from data_korean
def search_paths_names(dir):
    file_paths = []
    file_names = []
    files = os.listdir(dir)
    for file_ in files:
        file_names.append(file_)
        filename = os.path.join(dir,file_)
        file_paths.append(filename)
    return file_paths, file_names

file_paths, file_names = search_paths_names('./data_korean')

In [3]:
#2. convert xlsx files into csv files
for data in file_names:
    start, end = data.split('_')[4], data.split('_')[5].split('(')[0]
    data_name, csv = data.split('.')
    
    articles = pd.read_excel('./data_korean/' + data, encoding = 'cp949', converters={'ART_ID': str})

    if not(os.path.isdir('./csv_korean/' + start + '_' + end)):
        os.makedirs('./csv_korean/' + start + '_' + end)

    try:
        articles.to_csv('./csv_korean\\' + start + '_' + end + '\\' + data_name + ".csv", sep = ',', encoding = 'cp949', index = False)
    except UnicodeEncodeError:
        articles.to_csv('./csv_korean\\' + start + '_' + end + '\\' + data_name + ".csv", sep = ',', encoding = 'utf8', index = False)

In [4]:
file_paths = []
file_names = []

#3. Copy and Paste folder structure to articles_korean_with_date folder
def file_folder_split(dir):
    for x in os.listdir(dir):
        if os.path.isfile(os.path.join(dir, x)) and "csv" in x:
            file_paths.append(os.path.join(dir, x))
            file_names.append(x)
            #print("File ==> {0}".format(x))
        if os.path.isdir(os.path.join(dir, x)):
            new_dir = os.path.join(dir, x)
            new_dir = new_dir.replace("./csv_korean", "./articles_korean_with_date")
            if not(os.path.isdir(new_dir)):
                os.makedirs(new_dir)
            #print("Folder ==> {0}".format(x))
            file_folder_split(os.path.join(dir, x))

file_folder_split('./csv_korean')

In [6]:
for folders in os.listdir('./articles_korean_with_date'):
    for i in range(1, 13):
        if not(os.path.isdir(os.path.join('./articles_korean_with_date', folders) + '\\' + str(i))):
            os.makedirs(os.path.join('./articles_korean_with_date', folders) + '\\' + str(i))

In [7]:
#4. Divide each file by year and months

data_index = 0
total_index = 0
for data in file_paths:
    start, end = file_names[data_index].split('_')[4], file_names[data_index].split('_')[5].split('(')[0]

    try:
        articles = pd.read_csv(data, encoding = 'cp949', converters={'ART_ID': str})
    except UnicodeDecodeError:
        articles = pd.read_csv(data, encoding = 'utf8', converters={'ART_ID': str})
    articles = articles[['ART_ID', 'ART_CONTENT', 'ART_DATE']].dropna(axis = 0)
    articles = articles.reset_index()
    
    articles['DATE'] = pd.to_datetime(articles['ART_DATE'], format = '%Y%m%d')
    articles_with_date = articles[['ART_ID', 'ART_CONTENT', 'DATE']]
    articles_with_date['MONTH'] = pd.DatetimeIndex(articles_with_date['DATE']).month
    
    monthly_data = [None for i in range(12)]
    for i in range(0, 12):
        df = articles_with_date[articles_with_date['MONTH'] == (i+1)]
        if(df.empty):
            monthly_data[i] = None
        else:
            monthly_data[i] = df
    
    for i in range(0, 12):
        if monthly_data[i] is None:
            total_index += 1
            continue
        else:
            try:
                monthly_data[i].to_csv('./articles_korean_with_date\\'+ start + '_' + end + '\\' + str(i+1) + '\\'+ str(total_index) + '.csv', index = False, encoding = 'cp949')
            except UnicodeEncodeError:
                monthly_data[i].to_csv('./articles_korean_with_date\\'+ start + '_' + end + '\\' + str(i+1) + '\\'+ str(total_index) + '.csv', index = False, encoding = 'utf8')
            total_index += 1
    data_index += 1

C:\Users\YoungMinAhn\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [ ]:
#5. For further cleansing and collection.
#I will cleanse each data and create 'clenased_articles' column
#Then, vectorize the each file according to its month and year

from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Okt
okt = Okt()

file_paths = []
file_names = []

def file_folder_split(dir):
    for x in os.listdir(dir):
        if os.path.isfile(os.path.join(dir, x)) and "csv" in x:
            file_paths.append(os.path.join(dir, x))
            file_names.append(x)
        if os.path.isdir(os.path.join(dir, x)):
            file_folder_split(os.path.join(dir, x))
file_folder_split('./articles_korean_with_date')

data_index = 0
for data in file_paths:
    print(data)
    try:
        articles = pd.read_csv(data, encoding = 'cp949', converters={'ART_ID': str})
    except UnicodeDecodeError:
        articles = pd.read_csv(data, encoding = 'utf8', converters={'ART_ID': str})
    articles = articles[['ART_ID', 'ART_CONTENT', 'MONTH']].dropna(axis = 0)
    articles = articles.reset_index(drop = True)

    id = articles[['ART_ID', 'MONTH']]
    text = articles['ART_CONTENT'].values.tolist()

    # 1. For each sentence, imported the tagger(in this case, Twitter) and preprocessed.
    sentences_tag = []
    for sentence in text:
        morph = okt.pos(sentence, stem = True)
        sentences_tag.append(morph)

    # 2. Put in words that are noun/verb/adjective
    noun_adj_list = []
    for sentence1 in sentences_tag:
        words = ""
        for word, tag in sentence1:
            if tag in ['Noun','Adjective', 'Verb']:
                words += word + ", "
        noun_adj_list.append(words)

    noun_adj_df = pd.DataFrame(noun_adj_list, columns = ['ART_CONTENT_CLEAN'])
    # 3. Concated the ARTICLES_ID and saved the file.
    count_vec_df_concat = pd.concat([id, noun_adj_df], axis = 1)

    try:
        count_vec_df_concat.to_csv(data, index = False, encoding = 'cp949')
    except UnicodeEncodeError:
        count_vec_df_concat.to_csv(data, index = False, encoding = 'utf8')
    data_index += 1

In [30]:
temp = []
temp2 = []
def file_folder_split_temp(dir):
    data = []
    for x in os.listdir(dir):
        if os.path.isfile(os.path.join(dir, x)) and ("csv" in x):
            data.append(os.path.join(dir, x))
            temp2.append(x)
            #print("File ==> {0}".format(x))
        if os.path.isdir(os.path.join(dir, x)):
            #print("Folder ==> {0}".format(x))
            file_folder_split_temp(os.path.join(dir, x))
    
    if data:
        temp.append(data)
file_folder_split_temp('articles_korean_with_date')

In [ ]:
data_index = 0
for data_list in temp:
    partial_articles = []
    year = data_list[0].split('\\')[1]
    print(year)
    for data in data_list:
        try:
            articles = pd.read_csv(data, encoding = 'cp949', converters={'ART_ID': str})
        except UnicodeDecodeError:
            articles = pd.read_csv(data, encoding = 'utf8', converters={'ART_ID': str})
        
        partial_articles.append(articles)
        data_index += 1
    
    all_articles = partial_articles[0]
    for i in range(1, len(partial_articles)):
        all_articles = pd.concat([all_articles, partial_articles[i]])
    
    try:
        all_articles.to_csv('all_articles_korean_' + str(year) + '.csv', index = False, encoding = 'cp949')
    except UnicodeEncodeError:
        all_articles.to_csv('all_articles_korean_' + str(year) + '.csv', index = False, encoding = 'utf8')

    all_id = all_articles['ART_ID']
    month = int(all_articles['MONTH'].mean())

    vec = CountVectorizer()
    text_vec = vec.fit_transform(all_articles.ART_CONTENT_CLEAN.values.astype(str))
    count_vec_df = pd.DataFrame(text_vec.todense(), columns = vec.get_feature_names())
    all_id = all_id.reset_index(drop=True)
    yearly_vec = pd.concat([all_id, count_vec_df], axis = 1)
    
    try:
        yearly_vec.to_csv('all_articles_korean_vec_' + str(year) + "_" + str(month) + '.csv', index = False, encoding = 'cp949')
    except UnicodeEncodeError:
        yearly_vec.to_csv('all_articles_korean_vec_' + str(year) + "_" + str(month) + '.csv', index = False, encoding = 'utf8')